# Erzeugung von Pseudo-Zufallszahlen gemäß der Phasentypverteilung

Der folgende Code ist leicht abgewandelt den BuTools entnommen. Dort wird NumPy verwendet. Daher wurde der Code zusätzlich in eine NumPy-freie Variante überführt, so dass er später leichter in Java-Code übersetzt werden kann.

Quellen:

* https://en.wikipedia.org/wiki/Phase-type_distribution
* https://webspn.hit.bme.hu/~telek/tools/butools/index.php

## Einbinden der nötigen Bibliotheken

In [1]:
import numpy as np
from numpy.typing import ArrayLike
import numpy.matlib as ml
from math import log
from random import random

## Pseudo-Zufallszahlengenerator mit über globale Variable steuerbaren Werten

(Um später beide Varianten mit denselben [0,1)-Pseudo-Zufallszahlen versorgen zu können und so zu prüfen, ob beide Varianten identisch arbeiten.)

In [2]:
not_rnd_index=0
def get_not_rnd() -> float:
    values=[0.5, 0.15, 0.8, 0.3, 0.9, 0.05]
    global not_rnd_index
    value = values[not_rnd_index % len(values)]
    not_rnd_index += 1
    return value

## Generierung von Phasentyp-Zufallszahlen mit NumPy

In [3]:
def generatePhaseTypeRandom(S: ArrayLike , alpha: ArrayLike, rnd) -> float:
    """ Generate random variables from a phase-type distribution (using NumPy).
    Parameters
    ----------
    S : array_like
        Sub-intensity matrix of the phase-type distribution.
    alpha : array_like
        Initial distribution of the phase-type distribution.
    rnd : callable[[], float]
        A function that returns a random float in [0, 1).
    Returns
    -------
    float
        A random variable drawn from the specified phase-type distribution.
    Examples
    --------
    >>> S = [[-2, 1], [0, -3]]
    >>> alpha = [1, 0]
    >>> generatePhaseTypeRandom(S, alpha)
    """

    # Convert inputs to NumPy arrays
    S = np.array(S)
    alpha = np.array(alpha)
    n = S.shape[0]

    # Validate inputs
    if S.shape[1] != n:
        raise ValueError("Sub-intensity matrix S must be square.")
    if alpha.shape[0] != n:
        raise ValueError("Initial distribution alpha must have the same length as the size of S.")

    if alpha.shape != (n,):
        raise ValueError("Initial distribution alpha must be a one-dimensional array.")

    # Precompute cumulative distributions and sojourn times
    cumInitial = np.cumsum(alpha)
    sojourn = -1.0/np.diag(S)
    nextpr = ml.matrix(np.diag(sojourn))*S
    nextpr = nextpr - ml.matrix(np.diag(np.diag(nextpr)))
    nextpr = np.hstack((nextpr, 1.0-np.sum(nextpr,1)))
    nextpr = np.cumsum(nextpr,1)

    # Draw initial distribution
    r = rnd()
    state = 0
    while cumInitial[state]<=r:
        state+=1

    # Play state transitions
    time = 0
    while state<n:
        time += - np.log(rnd()) * sojourn[state]
        r = rnd()
        nstate = 0
        while nextpr[state,nstate]<=r:
            nstate += 1
        state = nstate
    return float(time)

## Generierung von Phasentyp-Zufallszahlen mit NumPy

(Weniger elegant, für die spätere Übersetung in Java-Code)

In [4]:
def generate_phase_type_random_no_numpy(S: list[list[float]] , alpha: list[float], rnd) -> float:
    """ Generate random variables from a phase-type distribution (do not use NumPy).
    Parameters
    ----------
    S : list[list[float]]
        Sub-intensity matrix of the phase-type distribution.
    alpha : alpha: list[float]
        Initial distribution of the phase-type distribution.
    rnd : callable[[], float]
        A function that returns a random float in [0, 1).
    Returns
    -------
    float
        A random variable drawn from the specified phase-type distribution.
    Examples
    --------
    >>> S = [[-2, 1], [0, -3]]
    >>> alpha = [1, 0]
    >>> generatePhaseTypeRandom(S, alpha)
    """

    # Determine size
    n = len(alpha)

    # Validate inputs
    if len(S) != len(S[0]):
        raise ValueError("Sub-intensity matrix S must be square.")
    if len(S) != n:
        raise ValueError("Initial distribution alpha must have the same length as the size of S.")
    if min(alpha) < 0:
        raise ValueError("Initial distribution alpha must have non-negative entries.")
    if abs(sum(alpha) - 1.0) > 1e-10:
        raise ValueError("Initial distribution alpha must sum to 1.")
    for i in range(n):
        if S[i][i] >= 0:
            raise ValueError("Diagonal entries of sub-intensity matrix S must be negative.")
        if any(S[i][j] < 0 for j in range(n) if j != i):
            raise ValueError("Off-diagonal entries of sub-intensity matrix S must be non-negative.")
        if sum(S[i]) > 0:
            raise ValueError("Row sums of sub-intensity matrix S must be less than or equal to zero.")

    # Precompute cumulative distributions and sojourn times
    cumInitial = [sum(alpha[0:i+1]) for i in range(n)]
    sojourn = [-1.0/S[i][i] for i in range(n)]
    nextpr = [[S[i][j]*sojourn[i] for j in range(n)] for i in range(n)]
    for i in range(n):
        nextpr[i][i] = 0.0
    nextprLast=[1.0 - sum(nextpr[i]) for i in range(n)]
    for i in range(n):
        nextpr[i].append(nextprLast[i])
    for i in range(n):
        nextpr[i]=[sum(nextpr[i][0:j+1]) for j in range(n+1)]

    # Draw initial distribution
    r = rnd()
    state = 0
    while cumInitial[state]<=r:
        state+=1

    # Play state transitions
    time = 0
    while state<n:
        time += - log(rnd()) * sojourn[state]
        r = rnd()
        nstate = 0
        while nextpr[state][nstate]<=r:
            nstate += 1
        state = nstate
    return float(time)

## Beispiel

In [5]:
# Example parameters
S = [[-2, 1.0], [0, -3]]
alpha = [1.0, 0.0]

### Zufallszahlen gemäß der beiden Methoden mit einem deterministischen Generator als Basis generieren

In [6]:
print("Erste Phasentyp-Pseudo-Zufallszahl (einmal mit und einmal ohne Numpy):")

not_rnd_index=0
print(generatePhaseTypeRandom(S, alpha, get_not_rnd))
not_rnd_index=0
print(generate_phase_type_random_no_numpy(S, alpha, get_not_rnd))

print("Zweite Phasentyp-Pseudo-Zufallszahl (einmal mit und einmal ohne Numpy):")

not_rnd_index=1
print(generatePhaseTypeRandom(S, alpha, get_not_rnd))
not_rnd_index=1
print(generate_phase_type_random_no_numpy(S, alpha, get_not_rnd))

Erste Phasentyp-Pseudo-Zufallszahl (einmal mit und einmal ohne Numpy):
0.9485599924429406
0.9485599924429406
Zweite Phasentyp-Pseudo-Zufallszahl (einmal mit und einmal ohne Numpy):
0.14669194754304693
0.14669194754304693


### Normales `random` verwenden

In [7]:
for _ in range(10):
    print(generate_phase_type_random_no_numpy(S, alpha, random))

0.6763434362295245
0.32077843584547866
0.16854317164559213
1.6831217389538038
0.37518310315706804
0.07384621822345463
0.06383468586035927
0.4058511444510991
0.4293458245296509
0.14476084011789545


In [8]:
value_sum = 0.0
square_sum = 0.0
count=100_000
for _ in range(count):
    value = generatePhaseTypeRandom(S, alpha, random)
    value_sum += value
    square_sum += value*value
print("Mean (with NumPy):", value_sum/count)
print("Variance (with NumPy):", square_sum/count - (value_sum/count)**2)

Mean (with NumPy): 0.6646360978693184
Variance (with NumPy): 0.32805066403580385
